# Pipeline for Data Preprocessing

## 0. Import and Configuration

In [10]:
import sys
import os
import pandas as pd
import importlib.util

# Import data cleaning module
sys.path.insert(0, os.path.abspath('./01_data_cleaning'))
spec = importlib.util.spec_from_file_location(
    "data_cleaning",
    "./01_data_cleaning/01_data_cleaning.py"
)
dc = importlib.util.module_from_spec(spec)
spec.loader.exec_module(dc)

# Import fraud relabeling module
spec = importlib.util.spec_from_file_location(
    "fraud_relabeling",
    "./02_fraud_relabeling/02_fraud_relabeling.py"
)
fr = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fr)

# Configure data cleaning
dc.ENABLE_RENAMING = True
dc.RAW_DIR = '../../data/raw'
dc.CLEANED_DIR = '../../data/cleaned'

# Configure fraud relabeling
fr.INPUT_DIR = '../../data/cleaned'
fr.OUTPUT_MEMBER_DIR = '../../data/by_member'
fr.OUTPUT_PROCESSED_DIR = '../../data/processed'
fr.CHUNKSIZE = 50000

# Create directories if they don't exist
directories = [
    dc.RAW_DIR,
    dc.CLEANED_DIR,
    fr.INPUT_DIR,
    fr.OUTPUT_MEMBER_DIR,
    fr.OUTPUT_PROCESSED_DIR
]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

## 1. Data Cleaning
This cell performs the following preprocessing tasks:
1. Standardize headers (e.g., "AccountID" → "Account ID")
2. Fix comma issues (remove extra commas in field values)
3. Clean Amount field (remove $ and commas, convert to numeric)
4. Fill missing values (Amount→0, others→"Unknown", "null"→empty)
5. Rename files based on date range (MM-DD-YYYY-MM-DD-YYYY.csv)

Output: Cleaned and renamed CSV files saved to ../../data/cleaned/


In [11]:
dc.main()

Raw: ../../data/raw
Cleaned: ../../data/cleaned

Found 4 CSV files in ../../data/raw

CSV Files List ⬇️
  1. TransactionData_2025Q3.csv (234.72 MB)
  2. TransactionData_2025Q2.csv (331.25 MB)
  3. TransactionData_2025Q1.csv (298.68 MB)
  4. TransactionData_2024Q4.csv (408.16 MB)

Processing Files...

[1/4] TransactionData_2025Q3.csv... Missing:76808, Newlines:1713394, →07-01-2025_to_09-01-2025.csv
[2/4] TransactionData_2025Q2.csv... Missing:110697, Newlines:2423101, →04-01-2025_to_06-30-2025.csv
[3/4] TransactionData_2025Q1.csv... Missing:100165, Newlines:2189448, →01-01-2025_to_03-31-2025.csv
[4/4] TransactionData_2024Q4.csv... Fields:10, Missing:138115, Newlines:2991607, Fraud_Merged:2, →09-01-2024_to_12-31-2024.csv

Processing Complete!


### Cleaned File Analysis

#### Size & Rows & Date Span

In [12]:
CLEANED_DIR = dc.CLEANED_DIR
csv_files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith('.csv')])

# Load all dataframes
dfs = {}
for filename in csv_files:
    dfs[filename] = pd.read_csv(os.path.join(CLEANED_DIR, filename))

# Collect stats
stats = []
for filename, df in dfs.items():
    df['Post Date'] = pd.to_datetime(df['Post Date'], errors='coerce')
    min_date = df['Post Date'].min()
    max_date = df['Post Date'].max()
    fraud_count = ((df['Fraud Adjustment Indicator'].notna()) &
                   (df['Fraud Adjustment Indicator'] != '')).sum()

    stats.append({
        'File': filename,
        'Rows': len(df),
        'Members': df['Member ID'].nunique(),
        'Date From': min_date.strftime('%m/%d/%Y') if pd.notna(min_date) else 'N/A',
        'Date To': max_date.strftime('%m/%d/%Y') if pd.notna(max_date) else 'N/A',
        'Days': (max_date - min_date).days if pd.notna(min_date) else 0,
        'Fraud %': round(fraud_count / len(df) * 100, 4) if len(df) > 0 else 0
    })

# Display table
df_stats = pd.DataFrame(stats)
display(df_stats)

# Summary
all_members = set()
for df in dfs.values():
    all_members.update(df['Member ID'].dropna())

total_fraud = sum([((dfs[f]['Fraud Adjustment Indicator'].notna()) &
                    (dfs[f]['Fraud Adjustment Indicator'] != '')).sum()
                   for f in dfs.keys()])

print(f"\nTotal Rows: {df_stats['Rows'].sum():,}")
print(f"Total Unique Members: {len(all_members):,}")
print(f"Total Fraud Indicators: {total_fraud:,}")
print(f"Overall Fraud %: {round(total_fraud / df_stats['Rows'].sum() * 100, 4)}%")

,File,Rows,Members,Date From,Date To,Days,Fraud %
0,01-01-2025_to_03-31-2025.csv,2189448,25572,01/01/2025,03/31/2025,89,0.0153
1,04-01-2025_to_06-30-2025.csv,2423101,27661,04/01/2025,06/30/2025,90,0.0111
2,07-01-2025_to_09-01-2025.csv,1713394,24074,07/01/2025,09/01/2025,62,0.0134
3,09-01-2024_to_12-31-2024.csv,2991606,26013,09/01/2024,12/31/2024,121,0.0098



Total Rows: 9,317,549
Total Unique Members: 29,656
Total Fraud Indicators: 1,124
Overall Fraud %: 0.0121%


#### Overlapping detection

In [13]:
CLEANED_DIR = dc.CLEANED_DIR
csv_files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith('.csv')])

print(f"Loading {len(csv_files)} files...")

# Load files and create row IDs
file_rows = {}
for filename in csv_files:
    df = pd.read_csv(os.path.join(CLEANED_DIR, filename))
    row_ids = set(df['Account ID'].astype(str) + '|' +
                  df['Member ID'].astype(str) + '|' +
                  df['Post Date'].astype(str) + '|' +
                  df['Post Time'].astype(str) + '|' +
                  df['Amount'].astype(str))
    file_rows[filename] = row_ids

# Calculate pairwise overlaps
results = []
for i, file1 in enumerate(csv_files):
    for j, file2 in enumerate(csv_files):
        if i < j:
            overlap = len(file_rows[file1] & file_rows[file2])
            pct1 = overlap / len(file_rows[file1]) * 100
            pct2 = overlap / len(file_rows[file2]) * 100

            results.append({
                'File 1': file1,
                'File 2': file2,
                'Overlap Rows': overlap,
                '% of File 1': round(pct1, 1),
                '% of File 2': round(pct2, 1)
            })

df_results = pd.DataFrame(results)

# Display as styled DataFrame
display(df_results)

Loading 4 files...


,File 1,File 2,Overlap Rows,% of File 1,% of File 2
0,01-01-2025_to_03-31-2025.csv,04-01-2025_to_06-30-2025.csv,0,0.0,0.0
1,01-01-2025_to_03-31-2025.csv,07-01-2025_to_09-01-2025.csv,0,0.0,0.0
2,01-01-2025_to_03-31-2025.csv,09-01-2024_to_12-31-2024.csv,0,0.0,0.0
3,04-01-2025_to_06-30-2025.csv,07-01-2025_to_09-01-2025.csv,0,0.0,0.0
4,04-01-2025_to_06-30-2025.csv,09-01-2024_to_12-31-2024.csv,0,0.0,0.0
5,07-01-2025_to_09-01-2025.csv,09-01-2024_to_12-31-2024.csv,0,0.0,0.0


## 2. Fraud Matching and Re-lable
This cell performs fraud detection in two stages:

1. **Reorganize by Member**: Group all transactions by Member ID into individual files
2. **Match Fraud Adjustments**: Find and mark original fraudulent transactions for each refund record
   - Match by amount and date (extract from description or 30-day range)
   - Prevent duplicate matching
   - Categorize as matched/unmatched/no_fraud

Output: Processed member files saved to `../../data/processed/[matched|unmatched|no_fraud]/`

### Stage 1: Reorganize transactions by member

In [16]:
num_members = fr.run_stage1()

STAGE 1: DATA REORGANIZATION
Input: ../../data/cleaned
Output: ../../data/by_member

Found 4 files
Processing 1/4: 01-01-2025_to_03-31-2025.csv
Processing 2/4: 04-01-2025_to_06-30-2025.csv
Processing 3/4: 07-01-2025_to_09-01-2025.csv
Processing 4/4: 09-01-2024_to_12-31-2024.csv
Created 29656 member files
Sorting files...
  Sorted 1000/29656 files
  Sorted 2000/29656 files
  Sorted 3000/29656 files
  Sorted 4000/29656 files
  Sorted 5000/29656 files
  Sorted 6000/29656 files
  Sorted 7000/29656 files
  Sorted 8000/29656 files
  Sorted 9000/29656 files
  Sorted 10000/29656 files
  Sorted 11000/29656 files
  Sorted 12000/29656 files
  Sorted 13000/29656 files
  Sorted 14000/29656 files
  Sorted 15000/29656 files
  Sorted 16000/29656 files
  Sorted 17000/29656 files
  Sorted 18000/29656 files
  Sorted 19000/29656 files
  Sorted 20000/29656 files
  Sorted 21000/29656 files
  Sorted 22000/29656 files
  Sorted 23000/29656 files
  Sorted 24000/29656 files
  Sorted 25000/29656 files
  Sorted 26

In [18]:
from glob import glob

# Configuration
BY_MEMBER_DIR = '../../data/by_member'
n = 10  # Threshold

# Get all member files and count transactions
member_files = glob(os.path.join(BY_MEMBER_DIR, 'member_*.csv'))
counts = [len(pd.read_csv(f)) for f in member_files]

# Calculate statistics
total_count = len(counts)
above_n = sum(1 for c in counts if c >= n)
below_n = total_count - above_n
above_ratio = (above_n / total_count) * 100
below_ratio = (below_n / total_count) * 100

# Print results
print(f"Threshold set to: {n}")
print(f"Records >= {n}: {above_n:,} ({above_ratio:.2f}%)")
print(f"Records < {n}: {below_n:,} ({below_ratio:.2f}%)")

Threshold set to: 10
Records >= 10: 23,295 (78.55%)
Records < 10: 6,361 (21.45%)


### Stage 2: Fraud detection and matching

In [11]:
# Stage 2: Fraud detection with minimum history length filter
min_history_length = 10
stats = fr.run_stage2(min_history_length)

STAGE 2: FRAUD DETECTION
Input: ../../data/by_member
Output: ../../data/processed
Min History Length: 10

Found 29656 member files
Filtering: only processing members with >= 10 transactions
  Processed 100/29656 members
  Processed 200/29656 members
  Processed 300/29656 members
  Processed 400/29656 members
  Processed 500/29656 members
  Processed 600/29656 members
  Processed 700/29656 members
  Processed 800/29656 members
  Processed 900/29656 members
  Processed 1000/29656 members
  Processed 1100/29656 members
  Processed 1200/29656 members
  Processed 1300/29656 members
  Processed 1400/29656 members
  Processed 1500/29656 members
  Processed 1600/29656 members
  Processed 1700/29656 members
  Processed 1800/29656 members
  Processed 1900/29656 members
  Processed 2000/29656 members
  Processed 2100/29656 members
  Processed 2200/29656 members
  Processed 2300/29656 members
  Processed 2400/29656 members
  Processed 2500/29656 members
  Processed 2600/29656 members
  Processed 2